<a href="https://colab.research.google.com/github/luguitar/ML_Books/blob/master/DIPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**`>LabDadosPrev("RPPS")` - Laboratório de Análise de Dados Previdenciários**

</br>

# **DIPR**
----
"_Sem dados, você é apenas mais uma pessoa com uma opinião_" - **W. E. Deming**

</br>

>[Marcos F Silva](https://audinalytics.netlify.app/)   
**Última Atualização:** 12/09/2021   
**Status:** em elaboração


# **Caracterização da base de dados**

A SPREV disponibiliza os dados relativos aos ingresssos e dispêndios de recursos dos RPPS num arquivo contendo os dados do **Demonstrativo de Informações Previdenciárias e Repasses - DIPR** demonstrativo no qual os RPPS repassam à SPREV essas informações.

Este é,sem dúvida, um dos principais arquivos de dados relacionados aos RPPS.

Cada arquivo contém os dados relativos a um exercício, de forma que cada arquivo de dados refere-se a um ano e contém os dados de todos os RPPS.

Estão disponíveis no site da SPrev os arquivos de dados do DIPR para os anos de 2014 em diante. 

Também é oportuno registrar que os nomes dos arquivos podem sofrer alterações de uma atualização para outra. Assim, pode ser que versões futuras dos arquivos de dados do DIPR tenham denominação distinta da que é apresentada neste documento.


# **Importação da base de dados**


In [ ]:
# Anexar o repositório do GitHub contendo os dados...
system('git clone https://github.com/marcosfs2006/LabDadosPrevRPPS.git')

In [ ]:
# Carregar os pacotes necessários...
options(scipen = 999, message = FALSE)
install.packages("writexl")
library(dplyr)
library(readr)
library(ggplot2)
library(stringr)
library(readxl)
library(lubridate)
library(purrr)
library(writexl)

A importação dos dados contidos nos arquivos pode ser feita da seguinte forma.

In [ ]:
# Importação dos dados
options(warn=-1)
suppressMessages({

arquivos_dipr <- c("LabDadosPrevRPPS/RG-OUTROS-DP-ENVIO_2021_20210821.xlsx",   
                   "LabDadosPrevRPPS/RG-OUTROS-DP-ENVIO_2020_20210528.xlsx",
                   "LabDadosPrevRPPS/RG-OUTROS-DP-ENVIO_2019_20210821.xlsx") 

# importação dos dados dos anos de 2019, 2020 e 2021 (até JUN/2021)
dipr <- map_dfr(arquivos_dipr,
                read_excel,
                sheet = "diprResultadoPrevidenciario",
                skip = 4,
                col_names = FALSE)

# Colocar nomes nas variáveis
names(dipr) <- c(
                    # 0. IDENTIFICAÇÃO DO DIPR
                    "ente", "uf", "competencia", "plano_segreg", "dt_info",
                    
                    # 1. BASES DE CÁLCULO DAS CONTRIBUIÇÕES DEVIDAS
                    "bc_pat_serv", "bc_pat_serv_ug", "bc_pat_apo", "bc_pat_pen",
                    "bc_serv", "bc_apo", "bc_pen",
                    
                    # 2. CONTRIBUIÇÕES REPASSADAS
                     "ct_pat_serv", "ct_pat_serv_ug", "ct_pat_apo", "ct_pat_pen",
                    "ct_serv", "ct_apo", "ct_pen",
                    
                    # 3. DEDUÇÕES
                    "deduc_pat", "deduc_pat_outros", "deduc_seg", "deduc_seg_outros",
                    
                    # 4. APORTES E TRANSFERÊNCIAS
                    "aportes_amortiz", "aportes_insufin", "aportes_despadm",
                    "aportes_pagtesouro", "aportes_outros",
                    
                    # 5. PARCELAMENTOS
                    "parcelamentos",
                    
                    # 6. BASES DE CÁLCULO DAS CONTRIBUIÇÕES DEVIDAS RELATIVAS À FOLHA DA UNIDADE GESTORA
                    "bcug_pat", "bcug_serv", "bcug_serv_afast", "bcug_apos", "bcug_pen",
                    
                    # 7. CONTRIBUIÇÕES ARRECADADAS PELA UNIDADE GESTORA 
                    "ctug_pat", "ctug_serv", "ctug_serv_afast", "ctug_apos", "ctug_pen",
                    
                    # 8. REMUNERAÇÃO BRUTA (SOMATÓRIO DO ENTE E DA UNIDADE GESTORA)
                    "rb_serv", "rb_apos", "rb_pen",
                     
                    # 9. NÚMERO DE BENEFICIÁRIOS (SOMATÓRIO DAS FOLHAS DO ENTE E DA UNIDADE GESTORA)  
                    "nb_serv", "nb_apos", "nb_pen",
                    "nb_dep",
                    
                    # 10. INGRESSOS DE RECURSOS
                    "ing_contr", "ing_aportes", "ing_parc", "ing_acres_leg", "ing_contr_cedlic",
                    "ing_comprev", "ing_aplicfin", "ing_ativos", "ing_outros",
                    
                    # 11. UTILIZAÇÃO DE RECURSOS
                    "desp_apo", "desp_pen_morte", "desp_auxdoenca", "desp_salmater",
                    "desp_salfam", "desp_auxreclu", "desp_decjud", "desp_beneftesouro",
                    "desp_despadm", "desp_invest", "desp_rest", "desp_comprev", 
                    "desp_outras",
                    
                    # 12. RESULTADO FINAL
                    "total_receita", "total_despesa", "resutado_final",
                    
                    # 13. MILITARES
                    "bcmil_pat_ativ", "bcmil_ativos", "bcmil_reserva", "bcmil_pen",
                    "ctmil_pat", "ctmil_ativos", "ctmil_reserva", "ctmil_pen",
                    "rbmil", "nbmil_ativos", "nbmil_reserva", "nbmil_pen", "nbmil_dep",
                    "ingmil_contr", "ingmil_aportes", "ingmil_outras", "despmil_reseva",
                    "despmil_penmorte", "despmil_outras", "resultado_final")

# Corrigir os nomes dos Entes na base de DIPR para coincidir com os nomes dos Entes
# no arquivo do regime previdenciário atual dos Entes.
dipr$ente <- gsub("^Prefeitura Municipal d[eoa] ", "", dipr$ente)

# Função para converter para formato numerico
convnum <- function(x){
  x <- gsub("\\.", "", x)
  x <- gsub(",", ".", x)
  as.numeric(x)
}

# convertendo os dados...
dipr <- as_tibble(map_at(.x = dipr,
                         .at = names(dipr)[-(1:5)],
                         .f = convnum))

# Converter a variável competencia para o formato data
# ESSE CÓDIGO NÃO FUNCIONA NO GOOGLE COLAB!!!!
#dipr <- dipr %>% 
#          mutate(competencia = dmy(paste("01", competencia, sep = "/")),
#                 dt_info = dmy(dt_info)) 

})

head(dipr, 2)

ente,uf,competencia,plano_segreg,dt_info,bc_pat_serv,bc_pat_serv_ug,bc_pat_apo,bc_pat_pen,bc_serv,⋯,nbmil_reserva,nbmil_pen,nbmil_dep,ingmil_contr,ingmil_aportes,ingmil_outras,despmil_reseva,despmil_penmorte,despmil_outras,resultado_final
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Governo do Distrito Federal,DF,JAN/2021,Financeiro,22/03/2021,662645466,NA,NA,NA,662645466,⋯,NA,NA,NA,0.01,NA,NA,NA,NA,NA,0.01
Governo do Distrito Federal,DF,JAN/2021,Previdenciário,22/03/2021,23358233,NA,NA,NA,23358233,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Como pode ser visto acima, essa base de dados é bem extensa. Possui 91 variáveis, cujo conteúdo será explicitado a seguir, visto que a SPREV não disponibiliza, até o momento, dicionários para suas bases de dados.

**Nota** - A descrição das variáveis reflete a minha compreensão das mesmas com base na experiência de trabalho com essa base de dados.

Coluna    | Descrição   
----------|------------------------------------------------------------------
**ente** | Nome do Ente
**uf** | Sigla da unidade da federação
**competencia** | Mês e ano de competência das receitas e despesas
**plano_segreg** | Indica se o ingresso ou dispêndio está vinculado ao plano previdenciário ou ao plano financeiro, Valores possíveis: "Financeiro", "Previdenciário"
**dt_info** | 
**bc_pat_serv** | Base de cálculo - contribuição patronal - relativa aos servidores ativos  
**bc_pat_serv_ug** | Base de cálculo - contribuição patronal - relativa aos servidores da unidade gestora do RPPS
**bc_pat_apo** | Base de cálculo - contribução patronal - relativa aos aposentados
**bc_pat_pen** | Base de cálculo - contribuição patronal - relativa aos pensionistas
**bc_serv** | Base de cálculo - contribuição do servidor ativo
**bc_apo** | Base de cálculo - contribuição do aposentado
**bc_pen** | Base de cálculo - contribuição do pensionista   
**ct_pat_serv** | Contribuição patronal repassada - relativa aos servidores ativos 
**ct_pat_serv_ug** | Contribuição patronal repassada - relativa a servidores afastados com benefícios pagos pela unidade gestora do RPPS (auxílios)
**ct_pat_apo** | Contribuição patronal repassada - relativa aos aposentados
**ct_pat_pen** | Contribuição patronal repassada - relativa aos pensionistas
**ct_serv** | Contribuição dos servidores ativos
**ct_apo** | Contribução dos aposentados
**ct_pen** | Contribuição dos pensionistas
**deduc_pat** | Deduções da contribuição patronal repassada - relativa benefícios pagos pelo Ente de responsabilidade do RPPS
**deduc_pat_outros** | Deduções da contribução patronal repassasa - outras deduções
**deduc_seg** | Dedução da contribuição do segurado - relativa benefícios pagos pelo Ente de responsabilidade do RPPS
**deduc_seg_outros** | Dedução da contribução do segurado - outras deduções
**aportes_amortiz** | Aportes e transferências de recursos - amortização do déficit atuarial
**aportes_insufin** | Aportes e transferências de recursos - cobertura de insuficiência financeira
**aportes_despadm** | Aportes e transferência de recursos - cobertura de despesas administrativas
**aportes_pagtesouro** | Aportes e transferência de recursos - pagamento de benefícios de responsabilidade do tesouro
**aportes_outros** | Aportes e transferência de recursos - outros
**parcelamentos** | Ingressos de recursos oriundos de parcelamentos de débitos
**bcug_pat**
**bcug_serv**
**bcug_serv_afast**
**bcug_apos**
**bcug_pen**
**ctug_pat**
**ctug_serv**
**ctug_serv_afast**
**ctug_apos**
**ctug_pen**
**rb_serv**
**rb_apos**
**rb_pen**
**nb_serv** | Número de beneficiários - servidores
**nb_apos** | Número de beneficiários - aposentados
**nb_pen**  | Número de beneficiários - pensionistas 
**nb_dep**  | Número de beneficiários - dependentes
**ing_contr** | Ingresso de recursos - contribuições
**ing_aportes** | Ingresso de recursos - aportes
**ing_parc**   | Ingresso de recursos - parcelamentos
**ing_acres_leg** | Ingresso de recursos - acréscimos legais de contribuições
**ing_contr_cedlic** | Ingresso de recursos - contribuições de servidores cedidos ou licenciados
**ing_comprev** | Ingresso de recursos - compensação financeira
**ing_aplicfin** | Ingresso de recursos - receitas líquidas de aplicações financeiras e investimentos
**ing_ativos** | Ingresso de recursos - rendimento dos demais ativos
**ing_outros** | Ingresso de recursos - outras receitas (10.9)
**desp_apo**
**desp_pen_morte**
**desp_auxdoenca**
**desp_salmater**
**desp_salfam**
**desp_auxreclu**
**desp_decjud**
**desp_beneftesouro**
**desp_despadm**
**desp_invest**
**desp_rest**
**desp_comprev**
**desp_outras**
**total_receita**
**total_despesa**
**resutado_final**
**bcmil_pat_ativ**
**bcmil_ativos**
**bcmil_reserva**
**bcmil_pen**
**ctmil_pat**
**ctmil_ativos**
**ctmil_reserva**
**ctmil_pen**
**rbmil**
**nbmil_ativos**
**nbmil_reserva**
**nbmil_pen**
**nbmil_dep**
**ingmil_contr**
**ingmil_aportes**
**ingmil_outras**
**despmil_reseva**
**despmil_penmorte**
**despmil_outras**
**resultado_final**



Essa base de dados é dividida em "blocos", conforme tentaremos mostrar a seguir:

Bloco | Descrição  | Variáveis incluídas no bloco
:----:|------------|----------------------------------------------------------
**0** | Variáveis de identificação do DIPR | `ente` , `uf`, `competencia`, `plano_segreg`, `dt_info`
**1** | Bases de cálculo das contribuições devidas |  `bc_pat_serv`, `bc_pat_serv_ug`, `bc_pat_apo`, `bc_pat_pen`, `bc_serv`, `bc_apo, bc_pen` 
**2** | Contribuições repassadas ao RPPS | `ct_pat_serv`, `ct_pat_serv_ug`, `ct_pat_apo`, `ct_pat_pen`, `ct_serv`, `ct_apo, ct_pen`
**3** | 


Essa base de dados não contém o CNPJ do Ente, fato que torna mais arriscado combinar esta base de dados com outras bases, visto que será necessáiro usar os nomes dos entes e a unidade da federação como chave.

# **Possibilidades de Análise**

A análise mais imediata que  pode ser realizada nessa base de dados é identificar quais foram os ingressos e desembolsos de recursos que o RPPS reportou ao longo do período analisado.

O primeiro passo é identificar as variáveis que se referem a ingressos e desembolsos de recursos. Uma vez identificadas essas variáveis, pode ser interessante verificar o comportamento delas ao longo do tempo com o objetivo de identificar possíveis valores não usuais.

Também pode ser interessante verificar quais RPPS não estão encaminhando o DIPR à SPREV.

Uma análise um pouco mais elaborada seria verificar a compatibilidade dos valores declarados dessas variáveis com valores informados na Matriz de Saldos Contábeis.

Essa é uma análise um pouco mais complexa e deixaremos de _standby_ até abordarmos como recuperar informações da MSC diretamente do SICONFI.

Vamos mostrar como identificar os registros da base que se referem a um determinado RPPS.

A primeira coisa a ser feita é identificar o RPPS que será objeto de análise. Para tanto, bastará informar a sigla da UF e o nome do Ente. Com isso podemos filtrar os dados desse RPPS. Exemplo:

In [ ]:
# Indique a sigla da UF do Ente que deseja pesquisar
UF <- "RJ" 

# Relação dos Entes nessa UF que encaminharam o DIPR no ano de 2020 à SPREV
ENTES <- dipr %>% filter(uf == UF) %>% pull(ente) %>% unique()
ENTES

[1] "Governo do Estado do Rio de Janeiro" "Angra dos Reis"                     
 [3] "Araruama"                            "Areal"                              
 [5] "Arraial do Cabo"                     "Barra Mansa"                        
 [7] "Bom Jardim"                          "Cambuci"                            
 [9] "Cantagalo"                           "Carapebus"                          
[11] "Cardoso Moreira"                     "Carmo"                              
[13] "Casimiro de Abreu"                   "Comendador Levy Gasparian"          
[15] "Conceição de Macabu"                 "Cordeiro"                           
[17] "Duas Barras"                         "Duque de Caxias"                    
[19] "Italva"                              "Itaocara"                           
[21] "Itatiaia"                            "Japeri"                             
[23] "Laje do Muriaé"                      "Macaé"                              
[25] "Mangaratiba"                         "Maricá"                             
[27] "Mendes"                              "Mesquita"                           
[29] "Miguel Pereira"                      "Miracema"                           
[31] "Natividade"                          "Nilópolis"                          
[33] "Niterói"                             "Nova Friburgo"                      
[35] "Nova Iguaçu"                         "Paraíba do Sul"                     
[37] "Paty do Alferes"                     "Petrópolis"                         
[39] "Pinheiral"                           "Piraí"                              
[41] "Porciúncula"                         "Quatis"                             
[43] "Queimados"                           "Quissamã"                           
[45] "Resende"                             "Rio Claro"                          
[47] "Rio das Ostras"                      "Santo Antônio de Pádua"             
[49] "São Fidélis"                         "São Gonçalo"                        
[51] "São João da Barra"                   "São José de Ubá"                    
[53] "São Pedro da Aldeia"                 "Sapucaia"                           
[55] "Saquarema"                           "Sumidouro"                          
[57] "Teresópolis"                         "Trajano de Moraes"                  
[59] "Valença"                             "Varre-Sai"                          
[61] "Vassouras"                           "Rio de Janeiro"                     
[63] "Armação dos Búzios"                  "Barra do Piraí"                     
[65] "Campos dos Goytacazes"               "Iguaba Grande"                      
[67] "Itaboraí"                            "Itaperuna"                          
[69] "Magé"                                "Rio Bonito"                         
[71] "São João de Meriti"                  "São Sebastião do Alto"              
[73] "Seropédica"                          "Silva Jardim"                       
[75] "Volta Redonda"                       "Aperibé"                            
[77] "Belford Roxo"                        "Cachoeiras de Macacu"               
[79] "Itaguaí"

Agora que sabemos quais Entes da UF selecionada estão disponíveis para análise e, importante, como se escreve o nome do Ente, podemos aplicar o filtro de forma mais segura. 

In [ ]:
ENTE_SELECIONADO <- 'Niterói'

# Filtro
dipr_ente <- dipr %>% filter(uf == UF, ente == ENTE_SELECIONADO)
head(dipr_ente)

ente,uf,competencia,plano_segreg,dt_info,bc_pat_serv,bc_pat_serv_ug,bc_pat_apo,bc_pat_pen,bc_serv,⋯,nbmil_reserva,nbmil_pen,nbmil_dep,ingmil_contr,ingmil_aportes,ingmil_outras,despmil_reseva,despmil_penmorte,despmil_outras,resultado_final
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Niterói,RJ,JAN/2021,Financeiro,28/04/2021,13242811,NA,NA,NA,13242811,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Niterói,RJ,JAN/2021,Previdenciário,28/04/2021,27958117,NA,NA,NA,27958117,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Niterói,RJ,FEV/2021,Financeiro,28/04/2021,13144762,NA,NA,NA,13144762,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Niterói,RJ,FEV/2021,Previdenciário,28/04/2021,28031309,NA,NA,NA,28031309,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Niterói,RJ,MAR/2021,Financeiro,17/05/2021,12955750,NA,NA,NA,12955750,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Niterói,RJ,MAR/2021,Previdenciário,17/05/2021,27860402,NA,NA,NA,27860402,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Considerando que existem variáveis que não possuem valores, seria bom retirá-las da base para que consigamos identificar exatamente quais variáveis foram utilizadas pelo RPPS selecionado.

Isso pode ser feito da seguinte forma:

In [ ]:
dipr_ente <- dipr_ente %>%
                      select_if(~!all(is.na(.))) %>%
                      arrange(plano_segreg)

head(dipr_ente)

ente,uf,competencia,plano_segreg,dt_info,bc_pat_serv,bc_serv,ct_pat_serv,ct_serv,aportes_amortiz,⋯,ing_outros,desp_apo,desp_pen_morte,desp_decjud,desp_despadm,desp_rest,desp_comprev,total_receita,total_despesa,resutado_final
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Niterói,RJ,JAN/2021,Financeiro,28/04/2021,13242811,13242811,2196982,1655351,NA,⋯,55375.60,19469732,8091794,19146.27,349839.1,2608.38,NA,4986094,27933120,-22947026
Niterói,RJ,FEV/2021,Financeiro,28/04/2021,13144762,13144762,2180716,1643095,NA,⋯,4594.29,19574032,8458498,10925.40,664116.1,966.82,36659.14,9704846,28745198,-19040351
Niterói,RJ,MAR/2021,Financeiro,17/05/2021,12955750,12955750,2149895,1619862,NA,⋯,12002573.60,19611218,8359165,NA,664132.0,16371.18,6027.83,18822454,28656914,-9834461
Niterói,RJ,ABR/2021,Financeiro,17/05/2021,12976304,12976304,2152766,1622031,NA,⋯,12502000.04,19673274,8323683,35633.79,747218.7,1047.64,63523.09,19320593,28844380,-9523787
Niterói,RJ,MAI/2021,Financeiro,29/07/2021,12948239,12948239,2148113,1618522,NA,⋯,13028199.55,19732684,8329688,16017.18,719709.3,29954.19,6771.08,19868174,28834824,-8966650
Niterói,RJ,JUN/2021,Financeiro,29/07/2021,12826124,12826124,2127854,1603258,NA,⋯,16804930.69,29454775,12532837,NA,959007.6,10651.06,6771.08,23617184,42964042,-19346859


In [ ]:
# Variáveis preenchidas
names(dipr_ente)

[1] "ente"             "uf"               "competencia"      "plano_segreg"    
 [5] "dt_info"          "bc_pat_serv"      "bc_serv"          "ct_pat_serv"     
 [9] "ct_serv"          "aportes_amortiz"  "aportes_insufin"  "aportes_despadm" 
[13] "parcelamentos"    "bcug_pat"         "bcug_serv"        "bcug_apos"       
[17] "bcug_pen"         "ctug_pat"         "ctug_serv"        "ctug_apos"       
[21] "ctug_pen"         "rb_serv"          "rb_apos"          "rb_pen"          
[25] "nb_serv"          "nb_apos"          "nb_pen"           "nb_dep"          
[29] "ing_contr"        "ing_aportes"      "ing_parc"         "ing_contr_cedlic"
[33] "ing_comprev"      "ing_aplicfin"     "ing_outros"       "desp_apo"        
[37] "desp_pen_morte"   "desp_decjud"      "desp_despadm"     "desp_rest"       
[41] "desp_comprev"     "total_receita"    "total_despesa"    "resutado_final"

As variáveis disponíveis para análise estão listadas acima. Deve-se, não obstante, observar que alguns RPPS apresentam segregação de massas e dessa forma será necessário separar os dados de cada plano.

Se o Ente que você está pesquisando enquadrar-se nessa situação, a separação pode ser feita da seguinte forma:

In [ ]:
# Código a ser executado caso o Ente apresente segregação de massas
dipr_ente_prev <- dipr_ente %>% filter(plano_segreg == "Previdenciário")
dipr_ente_fin  <- dipr_ente %>% filter(plano_segreg == "Financeiro")

In [ ]:
head(dipr_ente_prev)

ente,uf,competencia,plano_segreg,dt_info,bc_pat_serv,bc_serv,ct_pat_serv,ct_serv,aportes_amortiz,⋯,ing_outros,desp_apo,desp_pen_morte,desp_decjud,desp_despadm,desp_rest,desp_comprev,total_receita,total_despesa,resutado_final
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Niterói,RJ,JAN/2021,Previdenciário,28/04/2021,27958117,27958117,4638252,3494765,762096.1,⋯,NA,731957.2,209260.0,NA,NA,886.04,NA,8925762,942103.2,7983659
Niterói,RJ,FEV/2021,Previdenciário,28/04/2021,28031309,28031309,4650394,3503914,NA,⋯,NA,748424.0,207531.3,NA,NA,NA,NA,8184423,955955.3,7228468
Niterói,RJ,MAR/2021,Previdenciário,17/05/2021,27860402,27860402,4621505,3482136,NA,⋯,NA,755172.8,212065.5,NA,NA,1937.85,NA,8135225,969176.1,7166048
Niterói,RJ,ABR/2021,Previdenciário,17/05/2021,28267342,28267342,4689552,3533413,3143374.9,⋯,23326.33,778853.3,239331.3,NA,NA,483.16,NA,11422074,1018667.8,10403406
Niterói,RJ,MAI/2021,Previdenciário,29/07/2021,28069165,28069165,4656674,3508637,804494.1,⋯,107951.35,782523.5,216830.2,NA,NA,2807.64,NA,9113545,1002161.3,8111384
Niterói,RJ,JUN/2021,Previdenciário,29/07/2021,28078793,28078793,4658272,3509840,814495.4,⋯,10723.93,1208203.8,324725.0,NA,NA,3747.50,NA,9026984,1536676.3,7490307


In [ ]:
head(dipr_ente_fin)

ente,uf,competencia,plano_segreg,dt_info,bc_pat_serv,bc_serv,ct_pat_serv,ct_serv,aportes_amortiz,⋯,ing_outros,desp_apo,desp_pen_morte,desp_decjud,desp_despadm,desp_rest,desp_comprev,total_receita,total_despesa,resutado_final
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Niterói,RJ,JAN/2021,Financeiro,28/04/2021,13242811,13242811,2196982,1655351,NA,⋯,55375.60,19469732,8091794,19146.27,349839.1,2608.38,NA,4986094,27933120,-22947026
Niterói,RJ,FEV/2021,Financeiro,28/04/2021,13144762,13144762,2180716,1643095,NA,⋯,4594.29,19574032,8458498,10925.40,664116.1,966.82,36659.14,9704846,28745198,-19040351
Niterói,RJ,MAR/2021,Financeiro,17/05/2021,12955750,12955750,2149895,1619862,NA,⋯,12002573.60,19611218,8359165,NA,664132.0,16371.18,6027.83,18822454,28656914,-9834461
Niterói,RJ,ABR/2021,Financeiro,17/05/2021,12976304,12976304,2152766,1622031,NA,⋯,12502000.04,19673274,8323683,35633.79,747218.7,1047.64,63523.09,19320593,28844380,-9523787
Niterói,RJ,MAI/2021,Financeiro,29/07/2021,12948239,12948239,2148113,1618522,NA,⋯,13028199.55,19732684,8329688,16017.18,719709.3,29954.19,6771.08,19868174,28834824,-8966650
Niterói,RJ,JUN/2021,Financeiro,29/07/2021,12826124,12826124,2127854,1603258,NA,⋯,16804930.69,29454775,12532837,NA,959007.6,10651.06,6771.08,23617184,42964042,-19346859


Como pode ser visto nos quadros acima, algumas variáveis precisam ser processadas para que seja possível trabalhar de forma adequada com elas.

Por exemplo, a variável competência está num formato de caractere, os valores numéricos também estão no formato de texto.

Será necessário converter estas variáveis para os formatos adequados. 

No código a seguir será feito isso para toda a base de dados do DIPR (`dipr`).

In [ ]:
# Competências disponíveis na base de dados
unique(dipr$competencia)

[1] "JAN/2021" "FEV/2021" "MAR/2021" "ABR/2021" "MAI/2021" "JUN/2021"
 [7] "JAN/2020" "FEV/2020" "MAR/2020" "ABR/2020" "MAI/2020" "JUN/2020"
[13] "JUL/2020" "AGO/2020" "SET/2020" "OUT/2020" "NOV/2020" "DEZ/2020"
[19] "JAN/2019" "FEV/2019" "MAR/2019" "ABR/2019" "MAI/2019" "JUN/2019"
[25] "JUL/2019" "AGO/2019" "SET/2019" "OUT/2019" "NOV/2019" "DEZ/2019"

In [ ]:
# Alterar para abreviação em inglês... O Colab não aceita redefinição do locale. "Sys.setlocale()"
# FEB, APR, MAY, AUG, SEP, OCT,  DEC 

dipr <- dipr %>%
              mutate(competencia = str_replace_all(competencia, c("FEV" = "FEB",
                                                                  "ABR" = "APR",
                                                                  "MAI" = "MAY",
                                                                  "AGO" = "AUG",
                                                                  "SET" = "SEP",
                                                                  "OUT" = "OCT",
                                                                  "DEZ" = "DEC")))


# Converter a variável competencia para o formato data
dipr <- dipr %>% 
            mutate(competencia = dmy(paste("01", competencia, sep = "/")),
                   dt_info = dmy(dt_info)) 

head(dipr)

ente,uf,competencia,plano_segreg,dt_info,bc_pat_serv,bc_pat_serv_ug,bc_pat_apo,bc_pat_pen,bc_serv,⋯,nbmil_reserva,nbmil_pen,nbmil_dep,ingmil_contr,ingmil_aportes,ingmil_outras,despmil_reseva,despmil_penmorte,despmil_outras,resultado_final
<chr>,<chr>,<date>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Governo do Distrito Federal,DF,2021-01-01,Financeiro,2021-03-22,662645466,NA,NA,NA,662645466,⋯,NA,NA,NA,0.01,NA,NA,NA,NA,NA,0.01
Governo do Distrito Federal,DF,2021-01-01,Previdenciário,2021-03-22,23358233,NA,NA,NA,23358233,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Governo do Distrito Federal,DF,2021-02-01,Financeiro,2021-03-22,655813874,NA,NA,NA,655813874,⋯,NA,NA,NA,0.01,NA,NA,NA,NA,NA,0.01
Governo do Distrito Federal,DF,2021-02-01,Previdenciário,2021-03-22,23851522,NA,NA,NA,23851522,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Governo do Distrito Federal,DF,2021-03-01,Financeiro,2021-05-25,666327863,NA,NA,NA,666327863,⋯,NA,NA,NA,0.01,NA,NA,NA,NA,NA,0.01
Governo do Distrito Federal,DF,2021-03-01,Previdenciário,2021-05-25,25182111,NA,NA,NA,25182111,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


A base de dados está pronta para ser utilizada. Vamos exportar os dados para uma planilha.

In [ ]:
write_xlsx(dipr, "dipr_2019-2020-jul-ago-2021.xlsx")

Feitos estes ajustes na base de dados, podemos agora realizar operações sobre esses dados, como por exemplo, selecionar os registros relativos a um ente em particular, descartando as variáveis que não possuam valores, isto é, a colunas que não possuam valores.

In [ ]:
x <- dipr %>%
       filter(uf=="SP", ente == "Zacarias") %>%
       select_if(~!all(is.na(.)))

head(x)


ente,uf,competencia,plano_segreg,dt_info,bc_pat_serv,bc_pat_serv_ug,bc_serv,ct_pat_serv,ct_pat_serv_ug,⋯,desp_apo,desp_pen_morte,desp_auxdoenca,desp_salmater,desp_salfam,desp_despadm,desp_rest,total_receita,total_despesa,resutado_final
<chr>,<chr>,<date>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Zacarias,SP,2021-01-01,Previdenciário,2021-05-18,652391.6,NA,654448.9,104382.7,NA,⋯,98033.42,29213.73,NA,NA,NA,14127.36,NA,178374.7,141374.5,37000.19
Zacarias,SP,2021-02-01,Previdenciário,2021-05-18,655243.7,NA,657300.9,104839.0,NA,⋯,99734.56,29213.73,NA,NA,NA,15917.10,NA,179956.7,144865.4,35091.29
Zacarias,SP,2021-03-01,Previdenciário,2021-05-24,655699.6,NA,657756.8,104911.9,NA,⋯,99734.56,29213.73,NA,NA,NA,21934.75,NA,417655.6,150883.0,266772.53
Zacarias,SP,2021-04-01,Previdenciário,2021-05-24,652029.8,NA,654087.1,104509.0,NA,⋯,100916.82,29213.73,NA,NA,NA,30888.85,NA,1413939.4,161019.4,1252919.97
Zacarias,SP,2021-05-01,Previdenciário,2021-08-13,651551.4,NA,653608.7,104248.2,NA,⋯,102685.33,29213.73,NA,NA,NA,18299.97,NA,1762082.9,150199.0,1611883.89
Zacarias,SP,2021-06-01,Previdenciário,2021-08-13,647962.1,NA,648603.8,103673.9,NA,⋯,104652.91,29213.73,NA,NA,NA,18081.75,NA,255034.6,151948.4,103086.18
